Inteiro aleatório

In [ ]:
import secrets

def generate_num():
    # Gera um inteiro aleatório de 1024 bits
    n = secrets.randbits(1024)

    # Garante que ele tenha exatamente 1024 bits e seja ímpar:
    n |= (1 << 1023)    # define o bit mais significativo
    n |= 1              # define o bit 0 para torná-lo ímpar

    return n


def primes_up_to(n: int) -> list[int]:
    if n < 2: 
        return []
    
    is_prime = [True] * (n + 1)
    is_prime[0] = is_prime[1] = False

    # Marca múltiplos de cada primo como compostos
    lim = int(n**0.5)
    for p in range(2, lim + 1):
        if is_prime[p]:
            for multiple in range(p * p, n + 1, p):
                is_prime[multiple] = False

    # Coleta aqueles que permaneceram True
    return [i for i, prime in enumerate(is_prime) if prime]


def is_divisible_by_small_primes(n: int, primes: list[int]) -> bool:
    for p in primes:
        if p * p > n:
            break

        if n % p == 0:
            return n != p
        
    return False
    

def likely_prime_miller_rabin(n: int, k: int = 40):
    s, d = 0, n-1
    while d % 2 == 0:
        s += 1
        d //= 2

    for _ in range(k):
        a = secrets.randbelow(n-3) + 2  # base aleatória em [2, n-2]
        x = pow(a, d, n)
        if x in (1, n-1):
            continue
        for _ in range(s-1):
            x = pow(x, 2, n)
            if x == n-1:
                break
        else:
            return False
        
    return True

def generate_prime():
    primes_up_to_2000 = primes_up_to(2000)

    for _ in range(10_000):
        num = generate_num()
        if is_divisible_by_small_primes(num, primes_up_to_2000):
            continue

        if likely_prime_miller_rabin(num, 40):
            return num
        
    raise TimeoutError("Failed to generate prime after 50k iterations.")

In [33]:
num = generate_prime()
num

132969991645413115086731765141296196556073117841327563220755524396879773796629336394602678758629813152277058823390705691896160054431466674540597801714896636350325448064334899567341498138503752296158413913701894368368686943963066281660834317286923214700737732647223160305236049350625574239539860494953737927343